In [ ]:
import os
import cv2
import numpy as np
import gradio as gr
from sklearn.cluster import KMeans

In [ ]:
# Function to get the dominant color of an image using KMeans clustering
def get_dominant_color(image, k=4):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.reshape((image.shape[0] * image.shape[1], 3))

    clt = KMeans(n_clusters=k)
    clt.fit(image)

    num_labels = np.arange(0, len(np.unique(clt.labels_)) + 1)
    (hist, _) = np.histogram(clt.labels_, bins=num_labels)
    dominant_color = clt.cluster_centers_[np.argmax(hist)]

    return dominant_color

In [ ]:
# Function to calculate Euclidean distance between two colors
def color_distance(color1, color2):
    return np.sqrt(np.sum((color1 - color2) ** 2))

In [ ]:
# Function to recommend matching pants based on the shirt color, avoiding similar pants
def recommend_pants(shirt_image_path, pants_image_folder, threshold=50):
    try:
        shirt_image = cv2.imread(shirt_image_path)
        if shirt_image is None:
            return "Error: Could not load shirt image."
        
        shirt_dominant_color = get_dominant_color(shirt_image)
        pants_images = []
        pants_distances = []

        if not os.path.exists(pants_image_folder):
            return "Error: Pants image folder path does not exist."

        valid_extensions = ['.png', '.jpg', '.jpeg']
        for pant_name in os.listdir(pants_image_folder):
            if any(pant_name.lower().endswith(ext) for ext in valid_extensions):
                pant_path = os.path.join(pants_image_folder, pant_name)
                pant_img = cv2.imread(pant_path)
                if pant_img is None:
                    print(f"Error reading image: {pant_path}")
                    continue

                pants_images.append(pant_img)
                pant_dominant_color = get_dominant_color(pant_img)
                distance = color_distance(shirt_dominant_color, pant_dominant_color)
                pants_distances.append((pant_img, pant_dominant_color, distance))
        
        if not pants_images:
            return "Error: No valid pant images found in the specified folder."

        pants_distances.sort(key=lambda x: x[2])

        # Avoid same or similar pants colors
        top_pants = []
        added_colors = []
        
        for pant_img, pant_color, dist in pants_distances:
            if all(color_distance(pant_color, ac) > threshold for ac in added_colors):
                top_pants.append(cv2.cvtColor(pant_img, cv2.COLOR_BGR2RGB))
                added_colors.append(pant_color)
            if len(top_pants) == 3:
                break

        if len(top_pants) < 3:
            return "Error: Not enough distinct pants found."

        return top_pants
    
    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
# Function to load available pants images in the given folder

# GUI Interface
def display_pants(pants_folder_path):
    try:
        if not os.path.exists(pants_folder_path):
            return "Error: Folder does not exist."

        valid_extensions = ['.png', '.jpg', '.jpeg']
        pants_images = [f for f in os.listdir(pants_folder_path) if any(f.lower().endswith(ext) for ext in valid_extensions)]
        if len(pants_images) == 0:
            return "No pants images found in the folder."

        # Create image grid for display
        pants_image_grid = []
        for pant in pants_images:
            pant_path = os.path.join(pants_folder_path, pant)
            pant_img = cv2.imread(pant_path)
            if pant_img is not None:
                pants_image_grid.append(cv2.cvtColor(pant_img, cv2.COLOR_BGR2RGB))
        
        return pants_image_grid
    except Exception as e:
        return f"Error: {str(e)}"


In [ ]:
#gradio interface

def gradio_interface(shirt_image, pants_folder_path):
    shirt_image_path = "uploaded_shirt.jpg"
    cv2.imwrite(shirt_image_path, shirt_image)

    result = recommend_pants(shirt_image_path, pants_folder_path)
    return result


In [ ]:

# Main Gradio App Design
def main():
    with gr.Blocks(css=".gradio-container {background-color: #f0f0f5; font-family: 'Arial';}") as app:
        # Home Page
        with gr.Tab("Home"):
            gr.Markdown("""
    <div style="text-align: center;">
        <h1>👕 Shirt and Pants Matching Recommender 👖</h1>
        <p>Welcome to the professional matching recommender system.</p>
        <p>Our system helps you choose the best matching pants for your shirt using advanced color matching algorithms.</p>
        <h3>How it Works:</h3>
        <ol style="text-align: left; display: inline-block;">
            <li>Upload a shirt image or capture one live.</li>
            <li>Provide the folder containing pant images.</li>
            <li>Our algorithm will recommend the top 3 matching pants based on color similarity.</li>
        </ol>
        <p style="color: #555;">Try it now and discover the perfect outfit combinations!</p>
    </div>
    """)

        # Upload or Capture Shirt Image
        with gr.Tab("Shirt Upload"):
            gr.Markdown("### Upload or Capture Shirt Image")
            shirt_image = gr.Image(type="numpy", label="Upload Shirt Image")

        # Pants Folder Path
        with gr.Tab("Pants Collection"):
            gr.Markdown("### Available Pants Collection")
            pants_folder = gr.Textbox(label="Pants Folder Path", placeholder="Enter folder path containing pants images")
            display_pants_btn = gr.Button("Show Pants Collection")
            pants_collection = gr.Gallery(label="Pants Available", elem_id="pants-gallery")
            
            display_pants_btn.click(display_pants, inputs=[pants_folder], outputs=[pants_collection])

        # Button to get recommendations
        with gr.Tab("Recommendations"):
            gr.Markdown("### Get Top 3 Matching Pants")
            recommend_btn = gr.Button("Recommend Pants")
            result_pants = gr.Gallery(label="Top Pants Recommendations")
            recommend_btn.click(gradio_interface, inputs=[shirt_image, pants_folder], outputs=result_pants)

    app.launch(share=True)

if __name__ == "__main__":
    main()